# 楽曲のコード進行を取得したい。

## Songleを用いたやり方  

https://songle.jp/ にはAPIが公開されている  
http://widget.songle.jp/docs/v1 を読むと、  
http://widget.songle.jp/api/v1/song/chord.json?url=www.youtube.com/watch?v=PqJNc9KVIZE  
のようにすればコード進行のjsonを取得できるようだ。

REST API なのでHTTPメソッドでアクセスすればよく、pythonからはrequestsでデータを取得可能。  
https://note.nkmk.me/python-requests-web-api/

In [1]:
import requests
import pandas as pd
import json
import numpy as np

In [2]:
song_url = "www.youtube.com/watch?v=LtSNzPyo0lA"
# ぼくらはみんな意味不明 ピノキオピー

r_get = requests.get("http://widget.songle.jp/api/v1/song/chord.json?url=" + song_url)

print(r_get.status_code)

200


In [3]:
r_get.headers

{'Server': 'nginx', 'Date': 'Fri, 21 Dec 2018 15:20:10 GMT', 'Content-Type': 'application/json; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Status': '200 OK', 'Cache-Control': 'max-age=0, private, must-revalidate', 'Access-Control-Allow-Origin': '*', 'ETag': 'W/"d78bac811763b859e90b56a77a7894c4"', 'X-Frame-Options': 'SAMEORIGIN', 'X-XSS-Protection': '1; mode=block', 'X-Content-Type-Options': 'nosniff', 'X-Runtime': '0.379429', 'X-Request-Id': 'e013808b-d5b7-4b62-9b4f-0578490bb0c8', 'Content-Encoding': 'gzip'}

In [4]:
p = r_get.json()['chords'] 
# キー'chords'に対応する値を取得する

# 作戦会議

「ある曲の「ある部分の」転調したときのコード構成が他の曲の「ある部分の」コード構成と類似している」ことが知りたい  

・曲の区間を選び出す 方法は?→ 繰り返し構造がアノテートされているから、そこを拾えば行けそうだ感がある  
・そこのコードの音程を分析する  
    ・パース 原音とコード名 / N  
    ・音程の導出  
    ・分数コード(MVP段階では捨てても良い)  
    
・テンポ  
　　・基準となる数値を見つける  
  　・それぞれのdurationを基準値で割ってクオンタイズ  
    ・同時に、BPMを推定

・転調 は総当たりで行けるだろう、音程変化は楽

・コード構成の類似  durationが一致とは限らないので難しいが、基本的には構成音の共通がどれくらい多いかで見れば良さそう

# 作戦会議2

いや……一旦2曲の類似を確認するくらいならそんなに要らないんじゃないか?

・サビ同士を決め打ちで区間抽出して、決め打ちで転調して、コード構成の類似を見れば良い!

In [5]:
# 約62.640秒から約17.200秒間のコードを判別する
stt = json.dumps(p)

type(stt)

str

In [6]:


df00 = pd.read_json(stt)

# durationを1060で割ってみる
unit_ms = 1060
df00["duration"] = (df00["duration"]/ unit_ms).round().astype("int64")

In [7]:

df00.head()

,duration,index,name,start
0,3,0,N,0
1,2,1,Abm7,3283
2,1,2,C#m7,5411
3,1,3,F#,6467
4,2,4,Abm7,7533


In [8]:
key_to_half_tone = {
#         'Cb': 11, 
         'C': 0, 
         'C#': 1, 
         'Db': 1,
         'D' : 2,
         'D#': 3,
         'Eb': 3,
         'E' : 4,
#         'E#': 5,
#         'Fb': 4,
         'F' : 5,
         'F#': 6,
         'Gb': 6,
         'G' : 7,
         'G#': 8,
         'Ab': 8,
         'A' : 9,
         'A#': 10,
         'Bb': 10,
         'B' : 11,
#         'B#': 0,
        }

In [38]:
chord_to_half_tone = {
    '' : [0, 4, 7],  #メジャー
    'm' : [0, 3, 7],
    'aug' : [0, 4, 8],
    'dim' : [0, 3, 6],
    '7' : [0, 4, 7, 10],
    'M7' : [0, 4, 7, 11],
    'm7' : [0, 3, 7, 10],
    '6' : [0, 4, 7, 9],
}

In [36]:
import re

def chord_to_pitch(chord_name):
    # 文字列を受け取ると音の配列を返す
    # 0がC, 1がC#, ..., 11がB
    # chrod_nameが"N"ならばコードが無いので空配列
    if chord_name == "N":
        return np.array([], dtype='int32')
    # 一旦分数コードを考慮外にしたいので、 /以降を切り捨てる
    chord_name = re.search("^[^/]+", chord_name)
    # ^ 先頭、 ^/ スラッシュ以外、 + 一文字以上繰り返し
    # searchで返ってくるのはmatchオブジェクトなので、その中身をgroupで取得
    chord_name = chord_name.group()
    
    # キーと和音部分にパース
    if len(chord_name) > 1 and (chord_name[1] == "#" or chord_name[1] == "b"):
        key = chord_name[0:2]
        chord = chord_name[2:]
    else:
        key = chord_name[0]
        chord = chord_name[1:]
    
    #まずキーを数字に変換 ハッシュ(辞書)を使う
    key_num = key_to_half_tone[key]
    
    #次にコードを数字に変換 これもハッシュ(辞書)
    if chord not in chord_to_half_tone:
        print("ERROR! This chord is not registered: " + chord)
        return None
    
    chord_num = chord_to_half_tone[chord]

    return key_num + np.array(chord_num)


In [22]:
chord_to_pitch("E7")

array([ 4,  8, 11, 14])

array([], dtype=int32)

In [47]:
# 約62.640秒から約17.200秒間のコードを判別する
# 62.640 - 79.840 秒
# index 30以上39以下
sabi = df00.iloc[30:40]

In [48]:
sabi["chord_pitch"] = sabi["name"].map(chord_to_pitch)

c:\program files\python37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [49]:
sabi

,duration,index,name,start,chord_pitch
30,2,30,N,62750,[]
31,2,31,Abm,64878,"[8, 11, 15]"
32,2,32,N,67000,[]
33,2,33,Abm7,69122,"[8, 11, 15, 18]"
34,1,34,EM7,71250,"[4, 8, 11, 15]"
35,1,35,B/F#,72306,"[11, 15, 18]"
36,2,36,Abm,73367,"[8, 11, 15]"
37,1,37,EM7,75494,"[4, 8, 11, 15]"
38,1,38,F#,76550,"[6, 10, 13]"
39,2,39,Abm,77617,"[8, 11, 15]"


# youtubeとニコニコ動画で解析結果が大きく異なる問題について
妄想感傷代償連盟  
youtube https://www.youtube.com/watch?v=8pGRdRhjX3o  
nico https://www.nicovideo.jp/watch/sm30067009  

再生画面  
https://songle.jp/songs/www.youtube.com%2Fwatch%3Fv=8pGRdRhjX3o  
https://songle.jp/songs/www.nicovideo.jp%2Fwatch%2Fsm30067009  
youtubeの方にはほとんどコードが入っていない

REST APIを叩いた結果は違うのか?→全然違う。

http://widget.songle.jp/api/v1/song/chord.json?url=www.youtube.com/watch?v=8pGRdRhjX3o  
→62コード

http://widget.songle.jp/api/v1/song/chord.json?url=www.nicovideo.jp/watch/sm30067009  
→150コード

原因は不明……